In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2
from scipy.stats import beta
import pickle
from scipy.stats import pearsonr
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.neural_network import MLPClassifier,MLPRegressor
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,mean_squared_error
from itertools import combinations
from tabulate import tabulate

In [16]:
with open('glove.pkl','rb') as f:
    glove=pickle.load(f)
glove=2*glove-1
with open('sent_transformer.pkl','rb') as f:
    bert=pickle.load(f)
bert=2*bert-1
with open('universal_sentence_encoder.pkl','rb') as f:
    use=pickle.load(f)
use=2*use-1

In [17]:
def reliability(similarity):
    l=df.shape[0]
    scores=np.zeros(l,dtype=float)
    cnt=0
    simsum=0
    for i in range(124):
        x1=np.array(list(df[f'feature{i}']))
        for j in range(i+1,124):
            cnt=cnt+1
            x2=np.array(list(df[f'feature{j}']))
            s=similarity[i][j]
            simsum+=s
            rel=s*(1-np.abs(x1-x2))
            scores=scores+rel
    scores=scores/cnt
    return scores

In [18]:
df=pd.read_csv('CSV/Labelled_data.csv')
X=df.values
rel=reliability(bert)
print(rel)
l=X.shape[0]
limit=int(0.6*l)
scores=(rel-np.min(rel))/(np.max(rel)-np.min(rel))
X_train=X[:limit]
X_test=X[limit:]
train_scores=scores[:limit]
test_scores=scores[limit:]
augment_scale=10
new_train=[]
new_test=[]
for i in range(limit):
    scr=train_scores[i]
    freq=int(1+(augment_scale-1)*scr)
    for j in range(freq):
        new_train.append(X_train[i])
new_train=np.array(new_train)
np.random.shuffle(new_train)
for i in range(l-limit):
     scr=test_scores[i]
     freq=int(1+(augment_scale-1)*scr)
     for j in range(freq):
        new_test.append(X_test[i])
new_test=np.array(new_test)
np.random.shuffle(new_test)
X=np.vstack((new_train,X_test))
print(X.shape)
df=pd.DataFrame(X,columns=df.columns)
df.to_csv('CSV/bert.csv',index=False)

[-0.0061977  -0.11072458 -0.1636842  ... -0.09680749 -0.00093304
 -0.14803929]
(19657, 149)


In [19]:
new_train.shape

(17838, 149)

In [20]:
X_test.shape

(1819, 149)

In [21]:
l-limit

1819